In [1]:
# import libraries
from pyspark.sql import SparkSession

In [2]:
# create a spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

# access the Spark UI by navigating to localhost:4040

### GroupBy

In [3]:
# load green taxi trip data parquet files
df_green = spark.read.parquet('data/pq/green/*/*')

# register green taxi trip dataframe as a table
df_green.registerTempTable('green')

c:\Users\rebek\.virtualenvs\data-engineering-zoomcamp-erqhPacM\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
# execute a group by
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
# display the query results
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 03:00:00| 181|315.61000000000007|            14|
|2020-01-05 01:00:00|  42|163.10000000000005|            14|
|2020-01-06 13:00:00|  22|            337.62|             6|
|2020-01-21 14:00:00|  33|            571.46|            29|
|2020-01-06 14:00:00| 122|            110.26|             2|
|2020-01-19 17:00:00|  41| 488.8400000000001|            38|
|2020-01-08 09:00:00| 210|184.07999999999998|            10|
|2020-01-23 21:00:00| 129| 84.77999999999999|             9|
|2020-01-12 22:00:00| 181|284.01000000000005|            15|
|2020-01-05 16:00:00|  97|498.45000000000016|            39|
|2020-01-23 14:00:00| 185|             31.75|             2|
|2020-01-20 13:00:00| 166| 318.9400000000001|            24|
|2020-01-31 23:00:00|  76| 67.74000000000001|             2|
|2020-01-13 23:00:00| 22

In [6]:
# parition and write the query results
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [7]:
# load yellow taxi trip data parquet files
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

# register yellow taxi trip dataframe as a table
df_yellow.registerTempTable('yellow')

# execute a group by
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

# parition and write the query results
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

### OrderBy

In [8]:
# execute a group by followed by order by
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

# display the query results
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999997|            45|
|2020-01-01 00:00:00|  17|            195.03|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|175.67000000000002|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

### Joins

#### join two large tables

In [9]:
# read green and yellow revenue report parquet files
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [10]:
# add the service type as a prefix to 'amount' and 'number_records' fields in both dataframes
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
# join both the dataframes on 'hour' and 'zone' fields
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
# display the joined dataframe
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  36|295.34000000000003|                  11|            109.17|                    3|
|2020-01-01 00:00:00|  43|107.52000000000001|                   6|  6539.51000000001|                  390|
|2020-01-01 00:00:00|  68|              NULL|                NULL| 7825.070000000012|                  396|
|2020-01-01 00:00:00|  83|              94.1|                   7|               9.8|                    1|
|2020-01-01 00:00:00| 145|             65.14|                   4|            365.87|                   18|
|2020-01-01 00:00:00| 148|              NULL|                NULL| 6790.150000000003|                  371|
|2020-01-01 00:00:00| 159|  

In [13]:
# save the joined dataset as parquet file
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

#### join a large dataframe with a small dataframe

In [14]:
# read joined revenue report
df_join = spark.read.parquet('data/report/revenue/total')

In [15]:
# display the schema of the dataframe
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [16]:
# read taxi zones data
df_zones = spark.read.parquet('data/zones/')

In [17]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [18]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [19]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00| 769.7299999999997|                  45| 455.1700000000001|                   38|   Queens|   Boro Zone|
|2020-01-01 00:00:00|              NULL|                NULL|             42.41|                    2|   Queens|   Boro Zone|
|2020-01-01 00:00:00|              NULL|                NULL|              19.3|                    1| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|175.67000000000002|                   6|161.60999999999999|                    7| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00| 98.78999999999999|                   2|              NULL|                 NULL|   Queens|   Bor

In [21]:
df_result.drop('LocationID', 'zone').write.parquet('data/tmp/revenue-zones', mode='overwrite')